In [ ]:
#Step 1: Set Up the Environment
'''This installs TensorFlow, which is the main library we'll use for building,
training, and running our image classification model.
'''
!pip install tensorflow

In [ ]:
#Step 2: Import Libraries
'''tensorflow: The core library used for deep learning tasks.
layers, models: For building neural network layers and models.
MobileNetV2: A pre-trained model we'll use as a base.
ImageDataGenerator: To augment and preprocess the images.
to_categorical: Converts class labels to a one-hot encoding format.
'''

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import numpy as np

In [ ]:
#Step 3: Load and Preprocess the CIFAR-10 Dataset
'''
Loading the dataset: The CIFAR-10 dataset is available directly through Keras,
                    containing 60,000 32x32 color images in 10 different classes
                    (e.g., airplanes, cars, birds).
Normalization: We divide by 255.0 to scale pixel values between 0 and 1.
                This helps the model train faster and perform better.
One-hot encoding: Converts the labels into a format suitable for multi-class classification.
                For instance, label 3 will become [0, 0, 0, 1, 0, 0, 0, 0, 0, 0].
'''
# Load the dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalize the pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Check the shapes of data
print(f"x_train shape: {x_train.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")


In [ ]:
#Step 4: Data Augmentation
'''
Why use data augmentation?:
 It artificially expands the dataset by making slight modifications
 (e.g., rotating, shifting, flipping). This helps prevent overfitting,
  making the model more robust.
Parameters:
rotation_range: Rotates the images by up to 15 degrees.
width_shift_range and height_shift_range: Shifts the image horizontally and vertically
 by a fraction of the width/height.
horizontal_flip: Randomly flips images horizontally.
'''

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

datagen.fit(x_train)

In [ ]:
#Step 5: Set Up the Pre-trained Model
'''
Pre-trained models:
  Models like MobileNetV2 are already trained on a large dataset (ImageNet).
 By using them as a "base," you can leverage their feature extraction capabilities
 and speed up training.
Why freeze the base model (trainable = False)?
Since the pre-trained model already knows general image features,
we can save training time by only training the new layers we've added.
Adding custom layers:
GlobalAveragePooling2D: Reduces the dimensions of the feature maps,
                       keeping the most essential features.
Dense layers: Adds fully connected layers for classification.
              The last layer has 10 neurons (one for each CIFAR-10 class),
               using softmax for multi-class probability output.
'''
#Step 5: Set Up the Pre-trained Model
#1. Resize the Images to 224x224
# Resize images before training
x_train_resized = tf.image.resize(x_train, (224, 224))
x_test_resized = tf.image.resize(x_test, (224, 224))

# Check the shape to confirm resizing
print(f"x_train_resized shape: {x_train_resized.shape}")
print(f"x_test_resized shape: {x_test_resized.shape}")



In [ ]:
'''
Solution: Use Data Generators for On-the-Fly Resizing
Step-by-Step Implementation:
Use TensorFlow's ImageDataGenerator: This will resize images dynamically during training,
ensuring that only a batch of images is loaded into memory at a time, preventing crashes.

'''
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create a training ImageDataGenerator with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,         # Normalize pixel values to [0,1]
    rotation_range=15,        # Random rotation
    width_shift_range=0.1,    # Horizontal shift
    height_shift_range=0.1,   # Vertical shift
    horizontal_flip=True      # Randomly flip images horizontally
)

# Create a validation ImageDataGenerator with only rescaling
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Flow the training images from the array, with on-the-fly resizing to 224x224
train_generator = train_datagen.flow(x_train, y_train, batch_size=64, target_size=(224, 224))

# Flow the test images from the array, with on-the-fly resizing to 224x224
test_generator = test_datagen.flow(x_test, y_test, batch_size=64, target_size=(224, 224))


In [ ]:
'''
2.Set Up Your Model Using MobileNetV2: Now, use the pre-trained MobileNetV2 model with the appropriate input size:
'''
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

# Initialize MobileNetV2 with pre-trained weights
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

# Build the full model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Show a summary of the model
model.summary()


In [ ]:
'''
#Step 5: Set Up the Pre-trained Model
#2. Modify the Model Setup
# Set up MobileNetV2 with the correct input shape
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3)

# Freeze the base model layers
base_model.trainable = False

# Add custom layers on top of the base model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

# Show a summary of the model
model.summary()
'''

In [ ]:
#Step 5: Set Up the Pre-trained Model
#3. Update Training Code
#Make sure to use the resized datasets (x_train_resized, x_test_resized) when training:
# Use data augmentation for training
batch_size = 64
epochs = 20

history = model.fit(
    datagen.flow(x_train_resized, y_train, batch_size=batch_size),
    validation_data=(x_test_resized, y_test),
    steps_per_epoch=len(x_train) // batch_size,
    epochs=epochs
)

'''
Explanation:
Image Resizing: By resizing the CIFAR-10 images to 224x224, you ensure that they match the input shape expected by MobileNetV2, allowing the pre-trained weights to load properly.
Model Adjustments: Using the appropriate input_shape argument and keeping weights='imagenet' will utilize the pre-trained feature extractor effectively.
This approach should help you avoid the shape mismatch errors and leverage the pre-trained model correctly.
'''

In [ ]:
#Step 6: Compile the Model
'''optimizer='adam': Efficient optimizer that adjusts the learning rate during training.
loss='categorical_crossentropy': Appropriate loss function for multi-class classification.
metrics=['accuracy']: Monitors the accuracy during training.
'''
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
#Step 7: Train the Model
'''
Batch size: Number of images processed before updating the model's weights.
Epochs: One epoch means the model has seen the entire dataset once.
       More epochs allow the model to learn better.
datagen.flow: Uses augmented data during training.
validation_data: Evaluates the model's performance on unseen data.
'''
# Use data augmentation for training
batch_size = 64
epochs = 20

model.fit(
    datagen.flow(x_train, y_train, batch_size=batch_size),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // batch_size,
    epochs=epochs
)


In [ ]:
#Step 8: Evaluate the Model
'''
Evaluation: After training, this step tests how well the model performs on the test dataset.
Test accuracy: A measure of how many images the model correctly classifies.
'''
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'Test Accuracy: {test_acc}')


In [ ]:
#Step 9: Fine-tune the Model (Optional)
'''
Fine-tuning: Unfreezes the base model layers to improve performance
             by adjusting the pre-trained weights slightly.
Using a lower learning rate ensures that changes are small and controlled.
'''
# Unfreeze some layers of the base model
base_model.trainable = True

# Recompile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
history_fine = model.fit(
    datagen.flow(x_train, y_train, batch_size=batch_size),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // batch_size,
    epochs=10
)


In [ ]:
#Step 10: Save the Model
'''
Why save the model?
After training, you can save the model to use it later without retraining.
This .h5 file can be loaded and deployed anywhere.
'''

model.save('cifar10_classifier.h5')


In [ ]:
#
#

In [ ]:
#
#

In [ ]:
#
#